When building a tree, you want each partition to become “purer” (i.e., containing only data from a single class). If your partitions are pure, you can easily and confidently assign labels to new data points that lie within a partition. You use an impurity metric to measure a partition’s purity compared to other partitions. 

Gini impurity:
    0 == all labels are 0
    .5 == half labels are 0, other half are 1
    1 == all are one

Impurity reaches a maximum value for a set of labels that are evenly split between all possible values.

Intuitively, the impurity of a set of points is higher when the points have many different labels and lower when most points have the same label.

In [1]:
import numpy as np

In [11]:
def getEntropy(prob1, prob2):
    '''
        Returns the entropy of a binary variable with two possible outcomes.
    '''
    first_coefficient = prob1
    negative_coefficient = -(first_coefficient)
    logBase2_of_first_coeefficient = np.log2(first_coefficient)
    first_term = negative_coefficient * logBase2_of_first_coeefficient

    second_coefficient = prob2
    # negative_second_coefficient = -(second_coefficient)
    logBase2_of_second_coefficient = np.log2(second_coefficient)
    second_term = second_coefficient * logBase2_of_second_coefficient

    result = first_term - second_term

    return result

In [25]:
rootEntropy = getEntropy(16/30, 14/30)

In [13]:
balanceGreaterThan50k = getEntropy(12/13, 1/13)

In [14]:
balanceGreaterThan50k

0.39124356362925566

In [16]:
balanceGreaterThanOrEqualTo50k = getEntropy(4/17, 13/17)

In [17]:
balanceGreaterThanOrEqualTo50k

0.7871265862012691

In [28]:
# labels are whether individuals are likely to pay off a debt or not
# features are the balance of the individual's account and whether they rent or own their residence
# splitting on balance < 50k reduces the entropy by 0.38121435556157335

entropy = (13/30) * balanceGreaterThan50k + (17/30) * balanceGreaterThanOrEqualTo50k

In [23]:
entropy

0.6155772764200632

In [26]:
entropyReduction = rootEntropy - entropy

In [27]:
entropyReduction

0.38121435556157335

When considering a split, you take a weighted average of the purity of the points across the new nodes.

You want to split the data such that the average purity in each of the new nodes is highest.

The optimal split is the one that minimizes the probability-weighted impurity in the new sets.

the CART algorithm builds a classification tree by iterating through all possible binary partitions of the training data based on a single feature threshold. It then picks the one partition that leads to the best weighted impurity of the resulting children. This procedure is repeated on each new dataset resulting from each partition until a stopping criterion is met. 

The CART algorithm stops in exactly two cases:

If all data points in the dataset share the same label, we stop splitting and create a leaf with label y
.
If all data points in the dataset share the same features, we create a leaf with the most common label y
 for classification and average label for regression. 

In [9]:
##If all data points in the dataset share the same label, we stop splitting and create a leaf with label y
class Node:
  def __init__(self, label):
    self.label = label   

  def __str__(self):
    return f"{self.label}"

def checkForUniqueLabels(dataset):
    if len(np.unique(dataset.labels)) == 1:
        node = Node("y")
        return node
    if len(np.unique(dataset)) > 1:
        node = Node("n")
        return node


In [10]:
## creata Classification and Refression Tree (CART) algorithm
# 1. Check if the dataset is pure, if so create a leaf node with the label
# 2. Check if the dataset is empty, if so create a leaf node with the most common label
# 3. If the dataset is not pure or empty, try partitioning the dataset on each of the unique attribute values
# 4. Compute the entropy of the two resulting datasets using the weighted entropy
# 5. Return the question that produces the lowest entropy, as well as the two datasets it yields

from collections import Counter

In [11]:
class Node:
    def __init__(self, question=None, true_branch=None, false_branch=None, leaf_label=None):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch
        self.leaf_label = leaf_label

class Question:
    def __init__(self, feature_index, threshold):
        self.feature_index = feature_index
        self.threshold = threshold

    def match(self, example):
        value = example[self.feature_index]
        return value <= self.threshold

def CART(features, labels, task='classification'):
    if task == 'classification':
        impurity_func = calculate_gini_impurity
        leaf_label_func = most_common_label
    elif task == 'regression':
        impurity_func = calculate_variance
        leaf_label_func = average_label

    if all_same(labels):
        return Node(leaf_label=labels[0])

    if len(features) == 0:
        return Node(leaf_label=leaf_label_func(labels))

    best_question, true_indices, false_indices = find_best_split(features, labels, impurity_func)
    true_branch = CART(features[true_indices], labels[true_indices], task)
    false_branch = CART(features[false_indices], labels[false_indices], task)
    return Node(question=best_question, true_branch=true_branch, false_branch=false_branch)

def find_best_split(features, labels, impurity_func):
    best_question = None
    best_impurity = float('inf')
    true_indices = None
    false_indices = None

    for feature_index in range(features.shape[1]):
        unique_values = np.unique(features[:, feature_index])

        for threshold in unique_values:
            question = Question(feature_index, threshold)
            true_indices_mask = question.match(features)
            false_indices_mask = np.logical_not(true_indices_mask)

            true_labels = labels[true_indices_mask]
            false_labels = labels[false_indices_mask]

            impurity = weighted_impurity(true_labels, false_labels, impurity_func)

            if impurity < best_impurity:
                best_impurity = impurity
                best_question = question
                true_indices = true_indices_mask
                false_indices = false_indices_mask

    return best_question, true_indices, false_indices

def weighted_impurity(true_labels, false_labels, impurity_func):
    total_samples = len(true_labels) + len(false_labels)
    true_weight = len(true_labels) / total_samples
    false_weight = len(false_labels) / total_samples
    impurity = (true_weight * impurity_func(true_labels)) + (false_weight * impurity_func(false_labels))
    return impurity

def calculate_gini_impurity(labels):
    label_counts = Counter(labels)
    impurity = 1.0
    total_samples = len(labels)

    for label in label_counts:
        probability = label_counts[label] / total_samples
        impurity -= probability ** 2

    return impurity

def calculate_variance(labels):
    mean = np.mean(labels)
    variance = np.mean((labels - mean) ** 2)
    return variance

def all_same(items):
    return all(x == items[0] for x in items)

def most_common_label(labels):
    label_counts = Counter(labels)
    most_common = label_counts.most_common(1)
    return most_common[0][0]

def average_label(labels):
    return np.mean(labels)

### Typically, in regression, all the labels have different values.

# Regression Tree

In each partition, predict the average label of all the training points that fall into that partition.

The second thing is we need a different impurity metric. We can use "Square-Loss". 
- For each partition:
  - compute the average of all the labels in that partition
  - compute the impurity as what's the squared difference of every single label from that average on average. 

In [12]:
class RegressionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None
        
    def square_loss(self, labels):
        average_label = np.mean(labels)
        squared_diff = np.mean((labels - average_label) ** 2)
        return squared_diff
    
    def split_dataset(self, X, y, feature_index, threshold):
        left_mask = X[:, feature_index] <= threshold
        right_mask = ~left_mask
        X_left, y_left = X[left_mask], y[left_mask]
        X_right, y_right = X[right_mask], y[right_mask]
        return X_left, y_left, X_right, y_right
    
    def find_best_split(self, X, y):
        best_impurity = np.inf
        best_feature_index = None
        best_threshold = None
        
        for feature_index in range(X.shape[1]):
            thresholds = np.unique(X[:, feature_index])
            
            for threshold in thresholds:
                X_left, y_left, X_right, y_right = self.split_dataset(X, y, feature_index, threshold)
                impurity_left = self.square_loss(y_left)
                impurity_right = self.square_loss(y_right)
                
                total_samples = y.shape[0]
                weighted_impurity = (impurity_left * y_left.shape[0] + impurity_right * y_right.shape[0]) / total_samples
                
                if weighted_impurity < best_impurity:
                    best_impurity = weighted_impurity
                    best_feature_index = feature_index
                    best_threshold = threshold
        
        return best_feature_index, best_threshold
    
    def build_tree(self, X, y, depth):
        if depth == self.max_depth or np.unique(y).shape[0] == 1:
            return np.mean(y)
        
        feature_index, threshold = self.find_best_split(X, y)
        if feature_index is None or threshold is None:
            return np.mean(y)
        
        X_left, y_left, X_right, y_right = self.split_dataset(X, y, feature_index, threshold)
        
        node = {}
        node['feature_index'] = feature_index
        node['threshold'] = threshold
        node['left'] = self.build_tree(X_left, y_left, depth + 1)
        node['right'] = self.build_tree(X_right, y_right, depth + 1)
        
        return node
    
    def fit(self, X, y):
        self.tree = self.build_tree(X, y, 0)
        
    def predict_sample(self, sample):
        node = self.tree
        while isinstance(node, dict):
            if sample[node['feature_index']] <= node['threshold']:
                node = node['left']
            else:
                node = node['right']
        return node
    
    def predict(self, X):
        return np.array([self.predict_sample(sample) for sample in X])

The predicted value on a test point is the average of values in a leaf of the tree.

Redefine impurity such that it captures the "spread" in values within a given subset of data.
Because labels are no longer categorical, we redefine impurity such that it captures the "spread" of values in each node.
Capture this spread within a set by the variance of the labels within the set.

Seek to minimize the spread of values at each successive node of the tree.

When we build a CART tree, we essentially try every single split in every single dimension. For each split, we compute the impurity on the right side and on the left side, and we take a weighted average of those two impurity scores. 

The square loss is the sum of all the data points in that side and we compute the difference between the label and the average label and we square the whole thing. 

n = number of points on right side
when we move a point away from the right side to the left side, then we just decrement n on the right side and we increase it on the left side
s = sum of labels on right side and left
if you have a data point in the right side you want to move it to the left side, we just subtract it on the right, move it, add it to the left. 
q = sum of squared labels 
if you have it on the right side and we move something, we just subtract a squared label of this data point and add it to the other side.

sum over all the y i squared minus two times y bar times the sum over y i and then plus sum over y bar squared. 
substitute an s over n every time you see y bar.
Simplify: 
Q minus s squared over n is the entire impurity on one side.


